In [1]:
# INSTALL APPLICABLE PACKAGES ON COLAB RUNTIME

# #This section is base copy from google's sample import of tensorflow, keras, ploting and support libraries

#keras install libraries
!pip install -q keras
!pip install -q tqdm
!pip install imageio
!pip install -U scikit-learn

#openCV is used to process and filter images
#!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.21.2)


In [2]:
# IMPORT PYTHON LIBRARIES
# This cell contains all the references to libraries that are needed to train a convolutional neural network
from __future__ import print_function, division

import numpy as np
import random
import os
import glob
import copy
# https://opencv.org/
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2
import datetime
import pandas as pd
import time
import h5py
import csv
from os import listdir
from os.path import isfile, join

#from sklearn.cross_validation import KFold, train_test_split
from sklearn.metrics import log_loss, confusion_matrix
from sklearn.utils import shuffle
from sklearn import preprocessing

from PIL import Image, ImageChops, ImageOps

import matplotlib.pyplot as plt

from keras import backend as K
from keras.callbacks import EarlyStopping, Callback
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras import optimizers
from keras.models import Sequential, model_from_json
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, Activation, Dropout, Flatten, Dense

     |████████████████████████████████| 26.6MB 73.8MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


Using TensorFlow backend.


In [3]:
# MOUNT GOOGLE DRIVE
# Follow instructions as prompted after running code

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# PROCESS OUTPUT FILES INTO TRAINING EXAMPLES
# speedRange and aoaRange exists to filter-out noisy data
# mode is either one of these: 'stall', 'flutter', 'state'
def getData(Xdir, stallDir, speedRange, aoaRange, incSensors):
  allXdata = [f for f in listdir(Xdir)]
  lineCount = 215000 #lineCount=354000 for Feb 2019 experiments.
  sensors = set()
  speeds = set()
  aoas = set()
  
  dataCount = 0
  for file in allXdata:
    sensor = file.split("_")[0]
    speed = int(file.split("_")[1].split("m")[0])
    aoa = int(file.split("_")[2].split("d")[0])
    
    sensors.add(sensor)
    speeds.add(speed)
    aoas.add(aoa)    
    if speedRange[0]<=int(speed)<=speedRange[1] and aoaRange[0]<=int(aoa)<=aoaRange[1] and sensor in incSensors:
      dataCount += 1
  sensorCount = len(incSensors)
  sensors = sorted(sensors)
  speeds = sorted(speeds)
  aoas = sorted(aoas)
  
  examples = np.zeros((int(dataCount/sensorCount), sensorCount, lineCount))
  state = np.empty((int(dataCount/sensorCount)), dtype="S20")
  stall = np.empty((int(dataCount/sensorCount)), dtype="S20")
  
  stateCount=0
  for speed in speeds:
    for aoa in aoas:
      sensorCount=0
      for sensor in sensors:
        if speedRange[0]<=int(speed)<=speedRange[1] and aoaRange[0]<=int(aoa)<=aoaRange[1] and sensor in incSensors:
          print ("Currently processing sensor: %s, speed: %s, aoa: %s" %(sensor, speed, aoa))
          os.chdir(Xdir)
          file = sensor+"_"+str(speed)+"ms_"+str(aoa)+"deg.dat"
          examples[stateCount][sensorCount] = np.loadtxt(file)[0:lineCount]  
          os.chdir(stallDir)
          stall[stateCount] = str(np.loadtxt(file))
          state[stateCount] = str(speed)+"ms_"+str(aoa)+"deg"
          sensorCount += 1
      if speedRange[0]<=int(speed)<=speedRange[1] and aoaRange[0]<=int(aoa)<=aoaRange[1]:
        stateCount += 1

  print ("Finished processing all data.")
  return (examples, state, stall)

In [9]:
xfolder = "/content/gdrive/Shared drives/WindTunnelTests-Feb2019/Wind Tunnel Data 4/TrainingData/X.dat"
stallFolder = "/content/gdrive/Shared drives/WindTunnelTests-Feb2019/Wind Tunnel Data 4/TrainingData/Y.dat"
mySensors = ['PZTLB','PZTRB','PZTLT'] #Available sensors: PZTLB, PZTLT, PZTRB
(allExamples, allState, allStall) = getData (xfolder,stallFolder,speedRange=[0,30],aoaRange=[0,20], incSensors=mySensors)

Currently processing sensor: PZTLB, speed: 22, aoa: 0
Currently processing sensor: PZTLT, speed: 22, aoa: 0
Currently processing sensor: PZTRB, speed: 22, aoa: 0
Currently processing sensor: PZTLB, speed: 22, aoa: 2
Currently processing sensor: PZTLT, speed: 22, aoa: 2
Currently processing sensor: PZTRB, speed: 22, aoa: 2
Currently processing sensor: PZTLB, speed: 22, aoa: 4
Currently processing sensor: PZTLT, speed: 22, aoa: 4
Currently processing sensor: PZTRB, speed: 22, aoa: 4
Currently processing sensor: PZTLB, speed: 22, aoa: 6
Currently processing sensor: PZTLT, speed: 22, aoa: 6
Currently processing sensor: PZTRB, speed: 22, aoa: 6
Currently processing sensor: PZTLB, speed: 22, aoa: 8
Currently processing sensor: PZTLT, speed: 22, aoa: 8
Currently processing sensor: PZTRB, speed: 22, aoa: 8
Currently processing sensor: PZTLB, speed: 22, aoa: 10
Currently processing sensor: PZTLT, speed: 22, aoa: 10
Currently processing sensor: PZTRB, speed: 22, aoa: 10
Currently processing sens

In [0]:
# np.save('/content/gdrive/Team Drives/Wind Tunnel Tests - Feb2019/Wind Tunnel Data 2/NumpyFiles/allExamples_filtered.npy',allExamples)
# np.save('/content/gdrive/Team Drives/Wind Tunnel Tests - Feb2019/Wind Tunnel Data 2/NumpyFiles/allState_filtered.npy',allState)
# np.save('/content/gdrive/Team Drives/Wind Tunnel Tests - Feb2019/Wind Tunnel Data 2/NumpyFiles/allStall_filtered.npy',allStall)
# np.save('/content/gdrive/Team Drives/Wind Tunnel Tests - Feb2019/Wind Tunnel Data 2/NumpyFiles/allFlutter_filtered.npy',allFlutter)

In [0]:
# allExamples = np.load('/content/gdrive/Team Drives/WindTunnelTests-Feb2019/Wind Tunnel Data 2/NumpyFiles/allExamples.npy')
# allState = np.load('/content/gdrive/Team Drives/WindTunnelTests-Feb2019/Wind Tunnel Data 2/NumpyFiles/allState.npy')
# allStall = np.load('/content/gdrive/Team Drives/WindTunnelTests-Feb2019/Wind Tunnel Data 2/NumpyFiles/allStall.npy')
# allFlutter = np.load('/content/gdrive/Team Drives/WindTunnelTests-Feb2019/Wind Tunnel Data 2/NumpyFiles/allFlutter.npy')

In [0]:
# allExamples shape is (lineCount*len(allXdata), sonsorCount)

In [33]:
# List all data available in the database. Do not change this.
allSpeeds = sorted([22]) #Test2 Available speeds: [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 21, 22, 23, 24, 25, 26]
allAoas = sorted([0, 2, 4, 6, 8, 10, 12, 14, 16, 17, 18, 19, 20]) #Test2 Available aoas: [0, 2, 4, 6, 8, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20]
allSensors = sorted(['PZTLB', 'PZTLT', 'PZTRB']) #Test2 Available sensors: ['PZTLB', 'PZTLT', 'PZTRB', 'SG']

# The data we want to include in our model. Change this.
incSpeeds = sorted([22]) 
incAoas = sorted([0, 2, 4, 6, 8, 10, 12, 14, 16, 17, 18, 19, 20])
incSensors = sorted(['PZTLT']) 
bgData = 7143*3
endData = allExamples.shape[2] - 7143*3
totData = endData-bgData

# First identify where each example belongs to
i = 0
j = 0
stateMappings = dict()
sensorMappings = dict()
for speed in allSpeeds:
  for aoa in allAoas:
    stateMappings[(speed,aoa)] = i
    i += 1
for sensor in allSensors:
  sensorMappings[(sensor)] = j
  j += 1

# Then allocate model examples based on what we want.
stateToBeKept = list()
sensorToBeKept = list()
stateDict = dict()
i = 0
for speed in incSpeeds:
  for aoa in incAoas:
    stateToBeKept.append(stateMappings[(speed,aoa)])
    stateDict [i] = str(speed)+'m/s_'+str(aoa)+"deg"
    i += 1
for sensor in incSensors:
  sensorToBeKept.append(sensorMappings[(sensor)])

examples = allExamples[:,:,bgData:endData].copy()[np.ix_(stateToBeKept,sensorToBeKept,range(totData))]
state = allState.copy()[stateToBeKept]
stall = allStall.copy()[stateToBeKept]

# Standardize examples within each feature
meanDict = dict()
stdDict = dict()
for i in range(examples.shape[1]):
  mean = np.mean(examples[:,i,:])
  stddev = np.std(examples[:,i,:])
  meanDict[incSensors[i]] = mean
  stdDict[incSensors[i]] = stddev
  examples[:,i,:] = (examples[:,i,:]-mean)/stddev

print(examples.shape)

(13, 1, 172142)


In [34]:
print (meanDict)
print (stdDict)

{'PZTLT': 0.00012083879984061458}
{'PZTLT': 0.006812800279441543}


In [35]:
# Legend for stall & flutter: 0=no stall/flutter, 1=stall/flutter

dataCount = examples.shape[0]
sensorCount = examples.shape[1]
lineCount = examples.shape[2]
windowSize = 2800
splitparameter = lineCount//windowSize

# Prepare the data
examplesT = np.transpose(examples,[0,2,1])[:,0:windowSize*splitparameter,:]
trainingX = np.reshape(examplesT, (dataCount*splitparameter,windowSize,sensorCount))
trainingState = np.repeat(state, splitparameter, axis=0)
trainingStall = np.repeat(stall, splitparameter, axis=0)

print (trainingX.shape) #(number of states * splitparameter = number of examples, number of datapoints in each example = size, # of channels in each example)
print (trainingState.shape) #(number of examples)
print (trainingStall.shape)


(793, 2800, 1)
(793,)
(793,)


In [36]:
# Shuffle the data to remove any bias.
# This operation removes of the inter- & intra-state sequentiality.
def unison_shuffled_copies(b):
  p = np.random.permutation(len(b))
  return b[p], p

trainingX, p = unison_shuffled_copies(trainingX)
trainingState = trainingState[p]
trainingStall = trainingStall[p]

print (trainingX.shape)
print (trainingState.shape)

(793, 2800, 1)
(793,)


In [37]:
plt.plot(trainingX[300,0:2790,1])
plt.show()

IndexError: ignored

In [38]:
# SET UP CONVNET
# Importing the Keras libraries and packages
import pickle
import keras
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import BatchNormalization
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.metrics import sparse_categorical_accuracy
from keras import optimizers
from keras import regularizers
from keras import initializers
from keras import utils  

# convert data to categorical
encoder = preprocessing.LabelEncoder()

state_fit = encoder.fit(trainingStall)
stall_categorical = encoder.transform(trainingStall)
trainingY = stall_categorical ###Change this based on the output of interest###

# Initializing the CNN
classifier = Sequential()
REG = 1e-5
class_weight = {0:1., 1:4.}
leakyrelu = LeakyReLU(alpha=0.05)

# Conv layers
classifier.add(Conv1D(4, 100, input_shape=(windowSize,sensorCount), strides=1, activation='relu'))
classifier.add(MaxPooling1D())
classifier.add(Conv1D(6, 100, activation='relu'))
classifier.add(Conv1D(6, 100, activation='relu'))
classifier.add(MaxPooling1D())
classifier.add(Flatten())

# OUTPUT LAYER
classifier.add(Dense(max(trainingY)+1, activation="softmax"))

# Compiling the CNN
customAdam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
customAdamax = optimizers.Adamax(lr=0.0075, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
classifier.compile(optimizer = 'adamax', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# TRAIN THE MODEL
# Save the model weights after each epoch if the validation loss decreased
checkpointer = keras.callbacks.ModelCheckpoint(filepath='/content/gdrive/Shared drives/WindTunnelTests-Feb2019/Wind Tunnel Data 4/KerasFiles/tmpModel/tmpModel.hdf5', verbose=0, save_best_only=True)
history = classifier.fit(trainingX, trainingY, epochs = 200, batch_size=1024, verbose = 1, validation_split=0.15, shuffle=False, callbacks = [checkpointer])




Train on 674 samples, validate on 119 samples
Epoch 1/200
674/674 [==============================] - 1s 1ms/step - loss: 0.6594 - acc: 0.6499 - val_loss: 0.6801 - val_acc: 0.8319
Epoch 2/200
674/674 [==============================] - 0s 83us/step - loss: 0.9647 - acc: 0.7582 - val_loss: 0.4733 - val_acc: 0.8319
Epoch 3/200
674/674 [==============================] - 0s 83us/step - loss: 0.5684 - acc: 0.7582 - val_loss: 0.5621 - val_acc: 0.8319
Epoch 4/200
674/674 [==============================] - 0s 87us/step - loss: 0.5932 - acc: 0.7582 - val_loss: 0.6183 - val_acc: 0.8319
Epoch 5/200
674/674 [==============================] - 0s 85us/step - loss: 0.6301 - acc: 0.7582 - val_loss: 0.6361 - val_acc: 0.8067
Epoch 6/200
674/674 [==============================] - 0s 83us/step - loss: 0.6429 - acc: 0.7596 - val_loss: 0.6296 - val_acc: 0.8235
Epoch 7/200
674/674 [==============================] - 0s 81us/step - loss: 0.6394 - acc: 0.7582 - val_loss: 0.6070 - val_acc: 0.8319
Epoch 8/200
674/6

In [0]:
print (classifier.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_46 (Conv1D)           (None, 2800, 6)           3606      
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 2800, 6)           0         
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 2800, 6)           3606      
_________________________________________________________________
conv1d_48 (Conv1D)           (None, 2800, 6)           3606      
_________________________________________________________________
max_pooling1d_34 (MaxPooling (None, 1400, 6)           0         
_________________________________________________________________
conv1d_49 (Conv1D)           (None, 1400, 6)           3606      
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 700, 6)            0         
__________

In [0]:
classifier = keras.models.load_model('/content/gdrive/Team Drives/WindTunnelTests-Feb2019/Wind Tunnel Data 2/KerasFiles/06222019_training/try/tmpModel.hdf5')
print (classifier.outputs)

[<tf.Tensor 'dense_38_3/Softmax:0' shape=(?, 45) dtype=float32>]


In [0]:
print (trainingX.shape)

(4500, 2800, 2)


In [0]:
print (classifier.predict(trainingX[0:10,:,:]))

ValueError: ignored

In [0]:
# REFERENCE ARCHITECTURE
# convert data to categorical
encoder = preprocessing.LabelEncoder()
state_categorical = encoder.fit_transform(trainingState)
stall_categorical = encoder.fit_transform(trainingStall)
flutter_categorical = encoder.fit_transform(trainingFlutter)
trainingY = stall_categorical ###Change this based on the output of interest###

# Initializing the CNN
classifier = Sequential()
REG = 5e-5
class_weight = {0:1., 1:3.}
leakyrelu = LeakyReLU(alpha=0.05)

# Conv layers
classifier.add(Conv1D(6, 100, input_shape=(windowSize,sensorCount), strides=1))
classifier.add(leakyrelu)
classifier.add(MaxPooling1D())
classifier.add(Conv1D(6, 50, activation='relu'))
classifier.add(Conv1D(6, 50, activation='relu'))
classifier.add(MaxPooling1D())
classifier.add(Conv1D(6, 50, activation='relu'))
classifier.add(Conv1D(6, 50, activation='relu'))
classifier.add(MaxPooling1D())
classifier.add(Flatten())

# OUTPUT LAYER
classifier.add(Dense(max(trainingY)+1, activation="sigmoid"))

# Compiling the CNN
customAdam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
customAdamax = optimizers.Adamax(lr=0.0075, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
classifier.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# TRAIN THE MODEL
# Save the model weights after each epoch if the validation loss decreased
checkpointer = keras.callbacks.ModelCheckpoint(filepath='/content/gdrive/Team Drives/WindTunnelTests-Feb2019/Wind Tunnel Data 2/KerasFiles/06222019_training/tmpModel/tmpModel.hdf5', verbose=0, save_best_only=True)
history = classifier.fit(trainingX, trainingY, class_weight=class_weight, epochs = 150, batch_size=512, verbose = 1, validation_split=0.1, shuffle=False, callbacks = [checkpointer])


### V2 91% STATE ESTIMATION
# Conv layers
classifier.add(Conv1D(6, 300, input_shape=(windowSize,sensorCount), strides=1, padding='causal', dilation_rate=8, kernel_regularizer=regularizers.l2(REG)))
classifier.add(leakyrelu)
classifier.add(MaxPooling1D())
classifier.add(Conv1D(6, 150, activation='relu', padding='same', dilation_rate=4))
classifier.add(Conv1D(6, 150, activation='relu', padding='same', dilation_rate=4))
classifier.add(MaxPooling1D())
classifier.add(Conv1D(6, 150, activation='relu', padding='same'))
classifier.add(MaxPooling1D())
classifier.add(Flatten())

